In [1]:
#ppo algorithm single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from sklearn.preprocessing import StandardScaler

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate additional metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Train and evaluate the model with all metrics
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'MSFT_TRAINING.csv'
    test_file = 'MSFT_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize the PPO model and train
    model = PPO("MlpPolicy", env_train, verbose=1)
    model.learn(total_timesteps=100000)

    # Test the model on the training data
    obs, _ = env_train.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_train.step(action)

    # Generate report for the training session
    env_train.generate_report()

    # Calculate and display metrics for the training period
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Test the model on the testing data
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_test.step(action)

    # Generate report for the testing session
    env_test.generate_report()

    # Calculate and display metrics for the testing period
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")

# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1391 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1066        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015277362 |
|    clip_fraction        | 0.215       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -18.8       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0386     |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0146     |
|    value_loss         

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 8.05e+03     |
|    ep_rew_mean          | 3.17         |
| time/                   |              |
|    fps                  | 930          |
|    iterations           | 12           |
|    time_elapsed         | 26           |
|    total_timesteps      | 24576        |
| train/                  |              |
|    approx_kl            | 0.0073110294 |
|    clip_fraction        | 0.066        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.984       |
|    explained_variance   | -0.123       |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0017       |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.0102      |
|    value_loss           | 0.00182      |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 8.05e+03     |
|    ep_rew_mean          | 6.11         |
| time/                   |              |
|    fps                  | 903          |
|    iterations           | 22           |
|    time_elapsed         | 49           |
|    total_timesteps      | 45056        |
| train/                  |              |
|    approx_kl            | 0.0103186015 |
|    clip_fraction        | 0.092        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.834       |
|    explained_variance   | 0.152        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.00576     |
|    n_updates            | 210          |
|    policy_gradient_loss | -0.00934     |
|    value_loss           | 0.00377      |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.05e+03    |
|    ep_rew_mean          | 12.9        |
| time/                   |             |
|    fps                  | 879         |
|    iterations           | 32          |
|    time_elapsed         | 74          |
|    total_timesteps      | 65536       |
| train/                  |             |
|    approx_kl            | 0.008076161 |
|    clip_fraction        | 0.0958      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.844      |
|    explained_variance   | 0.76        |
|    learning_rate        | 0.0003      |
|    loss                 | 0.00338     |
|    n_updates            | 310         |
|    policy_gradient_loss | -0.00435    |
|    value_loss           | 0.00185     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.05e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.05e+03    |
|    ep_rew_mean          | 16.7        |
| time/                   |             |
|    fps                  | 842         |
|    iterations           | 42          |
|    time_elapsed         | 102         |
|    total_timesteps      | 86016       |
| train/                  |             |
|    approx_kl            | 0.010348206 |
|    clip_fraction        | 0.0982      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.593      |
|    explained_variance   | 0.402       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.00376    |
|    n_updates            | 410         |
|    policy_gradient_loss | -0.00489    |
|    value_loss           | 0.0043      |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.05e+


--- Agent Report ---
Agent starts with 0 holdings (neutral position), Initial Balance: 10000
Agent sells (short) at 336.7, Current Balance: 10000, Holdings: 1 Short
Agent closes short at 332.17, profit: 4.529999999999973, Current Balance: 10004.53, Holdings: 0
Agent sells (short) at 334.74, Current Balance: 10004.53, Holdings: 1 Short
Agent closes short at 334.89, profit: -0.14999999999997726, Current Balance: 10004.380000000001, Holdings: 0
Agent sells (short) at 334.65, Current Balance: 10004.380000000001, Holdings: 1 Short
Agent closes short at 328.45, profit: 6.199999999999989, Current Balance: 10010.580000000002, Holdings: 0
Agent sells (short) at 328.28, Current Balance: 10010.580000000002, Holdings: 1 Short
Agent closes short at 316.11, profit: 12.169999999999959, Current Balance: 10022.750000000002, Holdings: 0
Agent sells (short) at 316.34, Current Balance: 10022.750000000002, Holdings: 1 Short
Agent closes short at 314.78, profit: 1.5600000000000023, Current Balance: 10024.3

In [2]:
#dqn algorithm single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import DQN
from sklearn.preprocessing import StandardScaler

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate additional metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Train and evaluate the model with all metrics
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'MSFT_TRAINING.csv'
    test_file = 'MSFT_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize the DQN model and train
    model = DQN("MlpPolicy", env_train, verbose=1)
    model.learn(total_timesteps=100000)

    # Test the model on the training data
    obs, _ = env_train.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_train.step(action)

    # Generate report for the training session
    env_train.generate_report()

    # Calculate and display metrics for the training period
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Test the model on the testing data
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_test.step(action)

    # Generate report for the testing session
    env_test.generate_report()

    # Calculate and display metrics for the testing period
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")

# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.05e+03 |
|    ep_rew_mean      | 6.32     |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 818      |
|    time_elapsed     | 39       |
|    total_timesteps  | 32192    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000102 |
|    n_updates        | 8022     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 8.05e+03 |
|    ep_rew_mean      | 4.9      |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 845      |
|    time_elapsed     | 76       |
|    total_timesteps  | 64384    |
| train/              |        


--- Agent Report ---
Agent starts with 0 holdings (neutral position), Initial Balance: 10000
Agent sells (short) at 336.7, Current Balance: 10000, Holdings: 1 Short
Agent closes short at 316.39, profit: 20.310000000000002, Current Balance: 10020.31, Holdings: 0
Agent sells (short) at 314.94, Current Balance: 10020.31, Holdings: 1 Short
Agent closes short at 317.71, profit: -2.769999999999982, Current Balance: 10017.539999999999, Holdings: 0
Agent buys at 320.31, Current Balance: 10017.539999999999, Holdings: 1 Long
Agent closes long at 318.71, profit: -1.6000000000000227, Current Balance: 10015.939999999999, Holdings: 0
Agent buys at 313.6243, Current Balance: 10015.939999999999, Holdings: 1 Long
Agent closes long at 304.94, profit: -8.684300000000007, Current Balance: 10007.255699999998, Holdings: 0
Agent sells (short) at 305.7, Current Balance: 10007.255699999998, Holdings: 1 Short
Agent closes short at 308.31, profit: -2.6100000000000136, Current Balance: 10004.645699999997, Holdin

In [3]:
#a2c algorithm single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import A2C
from sklearn.preprocessing import StandardScaler

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate additional metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Train and evaluate the model with action logging for debugging
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'MSFT_TRAINING.csv'
    test_file = 'MSFT_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize the A2C model and train with more timesteps
    model = A2C("MlpPolicy", env_train, verbose=1)
    model.learn(total_timesteps=500000)  # Increased timesteps

    # Testing on the training data with action logging
    obs, _ = env_train.reset()
    done = False
    print("\n--- Training Session ---")
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_train.step(action)
        # Log each action and reward for debugging
        print(f"Step: {env_train.current_step}, Action: {action}, Reward: {reward}")

    # Generate report and metrics for the training session
    env_train.generate_report()
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Testing on the testing data with action logging
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    print("\n--- Testing Session ---")
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env_test.step(action)
        # Log each action and reward for debugging
        print(f"Step: {env_test.current_step}, Action: {action}, Reward: {reward}")

    # Generate report and metrics for the testing session
    env_test.generate_report()
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")



# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| time/                 |          |
|    fps                | 801      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.1     |
|    explained_variance | -6.65    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -0.0328  |
|    value_loss         | 0.00257  |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 805      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.06    |
|    explained_variance | 0.0322   |
|    learning_rate      | 0.0007   |
|    n_updates    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 8.05e+03  |
|    ep_rew_mean        | -0.866    |
| time/                 |           |
|    fps                | 699       |
|    iterations         | 1700      |
|    time_elapsed       | 12        |
|    total_timesteps    | 8500      |
| train/                |           |
|    entropy_loss       | -0.718    |
|    explained_variance | -2.12e+05 |
|    learning_rate      | 0.0007    |
|    n_updates          | 1699      |
|    policy_loss        | 0.0429    |
|    value_loss         | 0.00271   |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 8.05e+03  |
|    ep_rew_mean        | -0.866    |
| time/                 |           |
|    fps                | 699       |
|    iterations         | 1800      |
|    time_elapsed       | 12        |
|    total_timesteps    | 9000      |
| train/    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -0.866   |
| time/                 |          |
|    fps                | 705      |
|    iterations         | 3000     |
|    time_elapsed       | 21       |
|    total_timesteps    | 15000    |
| train/                |          |
|    entropy_loss       | -0.887   |
|    explained_variance | 0.0128   |
|    learning_rate      | 0.0007   |
|    n_updates          | 2999     |
|    policy_loss        | 0.00806  |
|    value_loss         | 0.000385 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -0.866   |
| time/                 |          |
|    fps                | 689      |
|    iterations         | 3100     |
|    time_elapsed       | 22       |
|    total_timesteps    | 15500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -0.243   |
| time/                 |          |
|    fps                | 686      |
|    iterations         | 4400     |
|    time_elapsed       | 32       |
|    total_timesteps    | 22000    |
| train/                |          |
|    entropy_loss       | -0.133   |
|    explained_variance | -0.821   |
|    learning_rate      | 0.0007   |
|    n_updates          | 4399     |
|    policy_loss        | 0.00632  |
|    value_loss         | 3.28e-05 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 8.05e+03  |
|    ep_rew_mean        | -0.243    |
| time/                 |           |
|    fps                | 688       |
|    iterations         | 4500      |
|    time_elapsed       | 32        |
|    total_timesteps    | 22500     |
| train/                |    

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | 0.0611   |
| time/                 |          |
|    fps                | 712      |
|    iterations         | 5700     |
|    time_elapsed       | 40       |
|    total_timesteps    | 28500    |
| train/                |          |
|    entropy_loss       | -0.601   |
|    explained_variance | -1.24    |
|    learning_rate      | 0.0007   |
|    n_updates          | 5699     |
|    policy_loss        | -0.0305  |
|    value_loss         | 0.0017   |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | 0.0611   |
| time/                 |          |
|    fps                | 714      |
|    iterations         | 5800     |
|    time_elapsed       | 40       |
|    total_timesteps    | 29000    |
| train/                |          |
|

...

Step: 2161, Action: 2, Reward: 0
Step: 2162, Action: 2, Reward: 0
Step: 2163, Action: 2, Reward: 0
Step: 2164, Action: 2, Reward: 0
Step: 2165, Action: 2, Reward: 0
Step: 2166, Action: 2, Reward: 0
Step: 2167, Action: 2, Reward: 0
Step: 2168, Action: 2, Reward: 0
Step: 2169, Action: 2, Reward: 0
Step: 2170, Action: 2, Reward: 0
Step: 2171, Action: 2, Reward: 0
Step: 2172, Action: 2, Reward: 0
Step: 2173, Action: 2, Reward: 0
Step: 2174, Action: 2, Reward: 0
Step: 2175, Action: 2, Reward: 0
Step: 2176, Action: 2, Reward: 0
Step: 2177, Action: 2, Reward: 0
Step: 2178, Action: 2, Reward: 0
Step: 2179, Action: 2, Reward: 0
Step: 2180, Action: 2, Reward: 0
Step: 2181, Action: 2, Reward: 0
Step: 2182, Action: 2, Reward: 0
Step: 2183, Action: 2, Reward: 0
Step: 2184, Action: 2, Reward: 0
Step: 2185, Action: 2, Reward: 0
Step: 2186, Action: 2, Reward: 0
Step: 2187, Action: 2, Reward: 0
Step: 2188, Action: 2, Reward: 0
Step: 2189, Action: 2, Reward: 0
Step: 2190, Action: 2, Reward: 0
Step: 2191

Step: 2572, Action: 2, Reward: 0
Step: 2573, Action: 2, Reward: 0
Step: 2574, Action: 2, Reward: 0
Step: 2575, Action: 2, Reward: 0
Step: 2576, Action: 2, Reward: 0
Step: 2577, Action: 2, Reward: 0
Step: 2578, Action: 2, Reward: 0
Step: 2579, Action: 2, Reward: 0
Step: 2580, Action: 2, Reward: 0
Step: 2581, Action: 2, Reward: 0
Step: 2582, Action: 2, Reward: 0
Step: 2583, Action: 2, Reward: 0
Step: 2584, Action: 2, Reward: 0
Step: 2585, Action: 2, Reward: 0
Step: 2586, Action: 2, Reward: 0
Step: 2587, Action: 2, Reward: 0
Step: 2588, Action: 2, Reward: 0
Step: 2589, Action: 2, Reward: 0
Step: 2590, Action: 2, Reward: 0
Step: 2591, Action: 2, Reward: 0
Step: 2592, Action: 2, Reward: 0
Step: 2593, Action: 2, Reward: 0
Step: 2594, Action: 2, Reward: 0
Step: 2595, Action: 2, Reward: 0
Step: 2596, Action: 2, Reward: 0
Step: 2597, Action: 2, Reward: 0
Step: 2598, Action: 2, Reward: 0
Step: 2599, Action: 2, Reward: 0
Step: 2600, Action: 2, Reward: 0
Step: 2601, Action: 2, Reward: 0
Step: 2602

Step: 2984, Action: 0, Reward: 0
Step: 2985, Action: 0, Reward: 0
Step: 2986, Action: 0, Reward: 0
Step: 2987, Action: 2, Reward: 0
Step: 2988, Action: 2, Reward: 0
Step: 2989, Action: 2, Reward: 0
Step: 2990, Action: 2, Reward: 0
Step: 2991, Action: 2, Reward: 0
Step: 2992, Action: 2, Reward: 0
Step: 2993, Action: 2, Reward: 0
Step: 2994, Action: 2, Reward: 0
Step: 2995, Action: 2, Reward: 0
Step: 2996, Action: 0, Reward: 0
Step: 2997, Action: 0, Reward: 0
Step: 2998, Action: 0, Reward: 0
Step: 2999, Action: 0, Reward: 0
Step: 3000, Action: 0, Reward: 0
Step: 3001, Action: 1, Reward: 1.9746135233630724
Step: 3002, Action: 1, Reward: 0
Step: 3003, Action: 2, Reward: 0.018619280841062423
Step: 3004, Action: 2, Reward: 0
Step: 3005, Action: 2, Reward: 0
Step: 3006, Action: 2, Reward: 0
Step: 3007, Action: 2, Reward: 0
Step: 3008, Action: 2, Reward: 0
Step: 3009, Action: 2, Reward: 0
Step: 3010, Action: 2, Reward: 0
Step: 3011, Action: 2, Reward: 0
Step: 3012, Action: 0, Reward: 0
Step: 3

Step: 3389, Action: 1, Reward: 0
Step: 3390, Action: 1, Reward: 0
Step: 3391, Action: 1, Reward: 0
Step: 3392, Action: 1, Reward: 0
Step: 3393, Action: 1, Reward: 0
Step: 3394, Action: 1, Reward: 0
Step: 3395, Action: 1, Reward: 0
Step: 3396, Action: 1, Reward: 0
Step: 3397, Action: 1, Reward: 0
Step: 3398, Action: 1, Reward: 0
Step: 3399, Action: 1, Reward: 0
Step: 3400, Action: 1, Reward: 0
Step: 3401, Action: 1, Reward: 0
Step: 3402, Action: 1, Reward: 0
Step: 3403, Action: 1, Reward: 0
Step: 3404, Action: 1, Reward: 0
Step: 3405, Action: 1, Reward: 0
Step: 3406, Action: 1, Reward: 0
Step: 3407, Action: 1, Reward: 0
Step: 3408, Action: 1, Reward: 0
Step: 3409, Action: 1, Reward: 0
Step: 3410, Action: 1, Reward: 0
Step: 3411, Action: 1, Reward: 0
Step: 3412, Action: 1, Reward: 0
Step: 3413, Action: 1, Reward: 0
Step: 3414, Action: 1, Reward: 0
Step: 3415, Action: 1, Reward: 0
Step: 3416, Action: 1, Reward: 0
Step: 3417, Action: 1, Reward: 0
Step: 3418, Action: 1, Reward: 0
Step: 3419

Step: 3800, Action: 2, Reward: 0
Step: 3801, Action: 2, Reward: 0
Step: 3802, Action: 2, Reward: 0
Step: 3803, Action: 2, Reward: 0
Step: 3804, Action: 2, Reward: 0
Step: 3805, Action: 2, Reward: 0
Step: 3806, Action: 2, Reward: 0
Step: 3807, Action: 2, Reward: 0
Step: 3808, Action: 2, Reward: 0
Step: 3809, Action: 2, Reward: 0
Step: 3810, Action: 2, Reward: 0
Step: 3811, Action: 2, Reward: 0
Step: 3812, Action: 2, Reward: 0
Step: 3813, Action: 2, Reward: 0
Step: 3814, Action: 2, Reward: 0
Step: 3815, Action: 2, Reward: 0
Step: 3816, Action: 2, Reward: 0
Step: 3817, Action: 2, Reward: 0
Step: 3818, Action: 2, Reward: 0
Step: 3819, Action: 2, Reward: 0
Step: 3820, Action: 2, Reward: 0
Step: 3821, Action: 2, Reward: 0
Step: 3822, Action: 2, Reward: 0
Step: 3823, Action: 2, Reward: 0
Step: 3824, Action: 2, Reward: 0
Step: 3825, Action: 2, Reward: 0
Step: 3826, Action: 2, Reward: 0
Step: 3827, Action: 2, Reward: 0
Step: 3828, Action: 2, Reward: 0
Step: 3829, Action: 2, Reward: 0
Step: 3830

In [4]:
#ensemble learning single agent
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO, DQN, A2C
from sklearn.preprocessing import StandardScaler
from collections import Counter

# Function to load and normalize data
def load_and_normalize_data(train_file, test_file):
    df_train = pd.read_csv(train_file)
    df_test = pd.read_csv(test_file)

    df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
    df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])

    scaler = StandardScaler()
    columns_to_normalize = ['open', 'high', 'low', 'close', 'volume']

    df_train[columns_to_normalize] = scaler.fit_transform(df_train[columns_to_normalize])
    df_test[columns_to_normalize] = scaler.transform(df_test[columns_to_normalize])

    return df_train, df_test, scaler

# Single-Agent Trading Environment
class SingleAgentEnv(gym.Env):
    def __init__(self, data, window_size=10, initial_balance=10000, scaler=None):
        super(SingleAgentEnv, self).__init__()
        self.data = data
        self.window_size = window_size
        self.current_step = 0
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 = neutral, 1 = long, -1 = short
        self.trades = []
        self.entry_price = 0
        self.log = []  # Log for detailed reporting
        self.scaler = scaler  # Store the scaler for inverse scaling

        # Action space: hold (0), buy (1), sell (2)
        self.action_space = spaces.Discrete(3)

        # Observation space: Stock prices (open, high, low, close, volume)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(window_size, 5), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.position = 0
        self.balance = self.initial_balance
        self.trades = []
        self.entry_price = 0
        self.log = []  # Reset log

        # Log initial holdings
        self.log.append(f"Agent starts with 0 holdings (neutral position), Initial Balance: {self.balance}")
        return self._get_observation(), {}

    def _get_observation(self):
        obs = self.data.iloc[self.current_step:self.current_step + self.window_size][['open', 'high', 'low', 'close', 'volume']].values.astype(np.float32)
        return obs

    def inverse_scale_price(self, price):
        # Inverse transform the scaled price to get the original value (considering the 'close' column)
        inverse_scaled = self.scaler.inverse_transform([[0, 0, 0, price, 0]])[0][3]
        return inverse_scaled

    def step(self, action):
        reward = 0
        current_price = self.data.iloc[self.current_step]['close']
        original_price = self.inverse_scale_price(current_price)  # Get original (inverse-scaled) price

        # If agent buys
        if action == 1:
            if self.position == 0:  # Only buy if neutral
                self.position = 1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent buys at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Long")
            elif self.position == -1:  # Close short position
                reward = self.entry_price - current_price  # Scaled reward
                original_reward = self.inverse_scale_price(self.entry_price) - original_price
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes short at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        # If agent sells
        elif action == 2:
            if self.position == 0:  # Only sell if neutral
                self.position = -1
                self.entry_price = current_price
                original_entry_price = self.inverse_scale_price(current_price)
                self.log.append(f"Agent sells (short) at {original_entry_price}, Current Balance: {self.balance}, Holdings: 1 Short")
            elif self.position == 1:  # Close long position
                reward = current_price - self.entry_price  # Scaled reward
                original_reward = original_price - self.inverse_scale_price(self.entry_price)
                self.balance += original_reward
                self.position = 0
                self.trades.append(original_reward)
                self.log.append(f"Agent closes long at {original_price}, profit: {original_reward}, Current Balance: {self.balance}, Holdings: 0")

        self.current_step += 1
        terminated = self.current_step >= len(self.data) - self.window_size
        truncated = False
        return self._get_observation(), reward, terminated, truncated, {}

    def generate_report(self):
        print("\n--- Agent Report ---")
        for log in self.log:
            print(log)
        print(f"Final Balance: {self.balance}")
        print(f"Total Profit: {self.balance - self.initial_balance}")
        print(f"Number of Holdings (Long): {1 if self.position == 1 else 0}, Short: {1 if self.position == -1 else 0}")
        print("-" * 40)

# Function to calculate metrics
def calculate_metrics(trades, initial_balance, final_balance):
    # Total Profit
    total_profit = final_balance - initial_balance

    # Cumulative Return
    cumulative_return = (final_balance - initial_balance) / initial_balance

    # Win Rate
    positive_trades = [trade for trade in trades if trade > 0]
    win_rate = len(positive_trades) / len(trades) if trades else 0

    # Profit Factor
    gross_profit = sum(trade for trade in trades if trade > 0)
    gross_loss = -sum(trade for trade in trades if trade < 0)
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf

    # Sharpe Ratio
    returns = np.array(trades)
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = mean_return / std_return if std_return != 0 else 0

    # Sortino Ratio (using only downside standard deviation)
    downside_std = np.std([min(0, r) for r in returns])
    sortino_ratio = mean_return / downside_std if downside_std != 0 else 0

    # Maximum Drawdown
    balance_series = np.cumsum([initial_balance] + trades)  # Series of balance over time
    peak_balance = np.maximum.accumulate(balance_series)
    drawdowns = (peak_balance - balance_series) / peak_balance
    max_drawdown = np.max(drawdowns) if drawdowns.size > 0 else 0

    metrics = {
        "Total Profit": total_profit,
        "Cumulative Return": cumulative_return,
        "Win Rate": win_rate,
        "Profit Factor": profit_factor,
        "Sharpe Ratio": sharpe_ratio,
        "Sortino Ratio": sortino_ratio,
        "Maximum Drawdown": max_drawdown
    }
    return metrics

# Ensemble model function
def ensemble_predict(actions):
    # Convert numpy arrays to integers for each action
    actions = [int(action) for action in actions]
    # Perform a majority vote among the actions (hold, buy, sell)
    action_counts = Counter(actions)
    return action_counts.most_common(1)[0][0]

# Train and evaluate the ensemble model
def train_and_evaluate():
    # Load and normalize the data
    train_file = 'MSFT_TRAINING.csv'
    test_file = 'MSFT_TESTING.csv'
    df_train_normalized, df_test_normalized, scaler = load_and_normalize_data(train_file, test_file)

    # Create the environment using the training data
    env_train = SingleAgentEnv(df_train_normalized, window_size=10, scaler=scaler)

    # Initialize each model and train them separately
    ppo_model = PPO("MlpPolicy", env_train, verbose=1)
    dqn_model = DQN("MlpPolicy", env_train, verbose=1)
    a2c_model = A2C("MlpPolicy", env_train, verbose=1)

    # Train each model
    ppo_model.learn(total_timesteps=50000)
    dqn_model.learn(total_timesteps=50000)
    a2c_model.learn(total_timesteps=50000)

    # Test the ensemble model on the training data
    obs, _ = env_train.reset()
    done = False
    while not done:
        # Get predictions from each model
        ppo_action, _ = ppo_model.predict(obs)
        dqn_action, _ = dqn_model.predict(obs)
        a2c_action, _ = a2c_model.predict(obs)

        # Aggregate the actions through majority voting
        final_action = ensemble_predict([ppo_action, dqn_action, a2c_action])

        # Step the environment with the final action
        obs, reward, done, truncated, info = env_train.step(final_action)

    # Calculate and display training metrics
    training_metrics = calculate_metrics(env_train.trades, env_train.initial_balance, env_train.balance)
    print("\n--- Training Metrics ---")
    for metric, value in training_metrics.items():
        print(f"{metric}: {value}")

    # Test the ensemble model on the testing data
    env_test = SingleAgentEnv(df_test_normalized, window_size=10, scaler=scaler)
    obs, _ = env_test.reset()
    done = False
    while not done:
        # Get predictions from each model
        ppo_action, _ = ppo_model.predict(obs)
        dqn_action, _ = dqn_model.predict(obs)
        a2c_action, _ = a2c_model.predict(obs)

        # Aggregate the actions through majority voting
        final_action = ensemble_predict([ppo_action, dqn_action, a2c_action])

        # Step the environment with the final action
        obs, reward, done, truncated, info = env_test.step(final_action)

    # Generate report for the testing session
    env_test.generate_report()

    # Calculate and display testing metrics
    testing_metrics = calculate_metrics(env_test.trades, env_test.initial_balance, env_test.balance)
    print("\n--- Testing Metrics ---")
    for metric, value in testing_metrics.items():
        print(f"{metric}: {value}")

# Run the training and evaluation
train_and_evaluate()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1540 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1395         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0076965587 |
|    clip_fraction        | 0.0474       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   | -7.59  

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 8.05e+03   |
|    ep_rew_mean          | 3.83       |
| time/                   |            |
|    fps                  | 1229       |
|    iterations           | 12         |
|    time_elapsed         | 19         |
|    total_timesteps      | 24576      |
| train/                  |            |
|    approx_kl            | 0.00782019 |
|    clip_fraction        | 0.0675     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.02      |
|    explained_variance   | -0.185     |
|    learning_rate        | 0.0003     |
|    loss                 | -0.00783   |
|    n_updates            | 110        |
|    policy_gradient_loss | -0.0111    |
|    value_loss           | 0.00178    |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.05e+03    |
|    ep_rew_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.05e+03    |
|    ep_rew_mean          | 8.94        |
| time/                   |             |
|    fps                  | 1244        |
|    iterations           | 22          |
|    time_elapsed         | 36          |
|    total_timesteps      | 45056       |
| train/                  |             |
|    approx_kl            | 0.006597498 |
|    clip_fraction        | 0.0979      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.905      |
|    explained_variance   | 0.478       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0206     |
|    n_updates            | 210         |
|    policy_gradient_loss | -0.00631    |
|    value_loss           | 0.00501     |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.05e+

------------------------------------
| time/                 |          |
|    fps                | 970      |
|    iterations         | 900      |
|    time_elapsed       | 4        |
|    total_timesteps    | 4500     |
| train/                |          |
|    entropy_loss       | -0.969   |
|    explained_variance | -18      |
|    learning_rate      | 0.0007   |
|    n_updates          | 899      |
|    policy_loss        | 0.00845  |
|    value_loss         | 0.000262 |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 968       |
|    iterations         | 1000      |
|    time_elapsed       | 5         |
|    total_timesteps    | 5000      |
| train/                |           |
|    entropy_loss       | -0.861    |
|    explained_variance | -1.85e+03 |
|    learning_rate      | 0.0007    |
|    n_updates          | 999       |
|    policy_loss        | -0.0424   |
|    value_loss         | 

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 8.05e+03  |
|    ep_rew_mean        | 1.05      |
| time/                 |           |
|    fps                | 969       |
|    iterations         | 2400      |
|    time_elapsed       | 12        |
|    total_timesteps    | 12000     |
| train/                |           |
|    entropy_loss       | -0.503    |
|    explained_variance | -4.96e+06 |
|    learning_rate      | 0.0007    |
|    n_updates          | 2399      |
|    policy_loss        | -0.00438  |
|    value_loss         | 0.000994  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | 1.05     |
| time/                 |          |
|    fps                | 969      |
|    iterations         | 2500     |
|    time_elapsed       | 12       |
|    total_timesteps    | 12500    |
| train/             

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -1.38    |
| time/                 |          |
|    fps                | 968      |
|    iterations         | 3700     |
|    time_elapsed       | 19       |
|    total_timesteps    | 18500    |
| train/                |          |
|    entropy_loss       | -0.326   |
|    explained_variance | -21.1    |
|    learning_rate      | 0.0007   |
|    n_updates          | 3699     |
|    policy_loss        | 0.000658 |
|    value_loss         | 0.000128 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -1.38    |
| time/                 |          |
|    fps                | 968      |
|    iterations         | 3800     |
|    time_elapsed       | 19       |
|    total_timesteps    | 19000    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -1.17    |
| time/                 |          |
|    fps                | 963      |
|    iterations         | 5000     |
|    time_elapsed       | 25       |
|    total_timesteps    | 25000    |
| train/                |          |
|    entropy_loss       | -0.247   |
|    explained_variance | -1.38    |
|    learning_rate      | 0.0007   |
|    n_updates          | 4999     |
|    policy_loss        | 3.14e-05 |
|    value_loss         | 4.65e-06 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -1.17    |
| time/                 |          |
|    fps                | 963      |
|    iterations         | 5100     |
|    time_elapsed       | 26       |
|    total_timesteps    | 25500    |
| train/                |          |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 8.05e+03  |
|    ep_rew_mean        | -1.17     |
| time/                 |           |
|    fps                | 957       |
|    iterations         | 6300      |
|    time_elapsed       | 32        |
|    total_timesteps    | 31500     |
| train/                |           |
|    entropy_loss       | -0.0355   |
|    explained_variance | -22       |
|    learning_rate      | 0.0007    |
|    n_updates          | 6299      |
|    policy_loss        | -7.91e-07 |
|    value_loss         | 6.41e-08  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -1.17    |
| time/                 |          |
|    fps                | 958      |
|    iterations         | 6400     |
|    time_elapsed       | 33       |
|    total_timesteps    | 32000    |
| train/             

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -1.04    |
| time/                 |          |
|    fps                | 958      |
|    iterations         | 7600     |
|    time_elapsed       | 39       |
|    total_timesteps    | 38000    |
| train/                |          |
|    entropy_loss       | -0.00464 |
|    explained_variance | 0.57     |
|    learning_rate      | 0.0007   |
|    n_updates          | 7599     |
|    policy_loss        | 5.27e-07 |
|    value_loss         | 1.24e-06 |
------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 8.05e+03  |
|    ep_rew_mean        | -1.04     |
| time/                 |           |
|    fps                | 958       |
|    iterations         | 7700      |
|    time_elapsed       | 40        |
|    total_timesteps    | 38500     |
| train/                |    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 8.05e+03  |
|    ep_rew_mean        | -0.778    |
| time/                 |           |
|    fps                | 961       |
|    iterations         | 8900      |
|    time_elapsed       | 46        |
|    total_timesteps    | 44500     |
| train/                |           |
|    entropy_loss       | -0.028    |
|    explained_variance | -331      |
|    learning_rate      | 0.0007    |
|    n_updates          | 8899      |
|    policy_loss        | -4.76e-05 |
|    value_loss         | 0.000174  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.05e+03 |
|    ep_rew_mean        | -0.778   |
| time/                 |          |
|    fps                | 961      |
|    iterations         | 9000     |
|    time_elapsed       | 46       |
|    total_timesteps    | 45000    |
| train/             